In [1]:
from keras.applications import VGG16

Using TensorFlow backend.


In [2]:
img_row=200
img_col=200
#reducing img size to 100x100 to reduce time

#### Loading model without top or Fully connected layers

In [3]:
model=VGG16(weights='imagenet',include_top=False,input_shape=(img_row,img_col,3))

### Printing layers

In [4]:
for (i,layer) in enumerate(model.layers):
    print(str(i)+" "+layer.__class__.__name__,layer.trainable)

0 InputLayer False
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


### Freeze layers

In [5]:
for layers in model.layers:
    layers.trainable=False

In [6]:
for (i,layer) in enumerate(model.layers):
    print(str(i)+" "+layer.__class__.__name__,layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [7]:
model.output

<tf.Tensor 'block5_pool/MaxPool:0' shape=(None, 6, 6, 512) dtype=float32>

### Construct fully connected top model

In [8]:
def add_top_model(bottom_model,num_classes,D=256):
    top_model=bottom_model.output
    top_model=Flatten(name='flatten')(top_model)
    top_model=Dense(512)(top_model)
    top_model=LeakyReLU(alpha=0.1)(top_model) #adding leakyrelu instead of using relu activation in Dense
    top_model=Dense(256)(top_model)
    top_model=LeakyReLU(alpha=0.1)(top_model)
    top_model=Dense(256)(top_model)
    top_model=LeakyReLU(alpha=0.1)(top_model)
    top_model=Dense(128)(top_model)
    top_model=LeakyReLU(alpha=0.1)(top_model)
    top_model=Dropout(0.3)(top_model)
    top_model=Dense(num_classes,activation='softmax')(top_model)
    
    return top_model

### Input layer specification

In [9]:
model.input

<tf.Tensor 'input_1:0' shape=(None, 200, 200, 3) dtype=float32>

### All layers specification

In [10]:
model.layers

In [11]:
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Activation,Flatten,LeakyReLU
from keras.layers import MaxPooling2D,Conv2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization

num_classes=10

In [12]:
FC_head=add_top_model(model,num_classes)

model_new=Model(inputs=model.input,output=FC_head) #Appending model and FC_head

/opt/anaconda3/envs/venv-3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
model_new.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0   

In [14]:
from keras.preprocessing.image import ImageDataGenerator

In [15]:
train_dir='Bollywood_Dataset/Bollywood_Dataset/train'
validation_dir='Bollywood_Dataset/Bollywood_Dataset/validation'
train_data_gen=ImageDataGenerator(rescale=1./255,rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  horizontal_flip=True,
                                  vertical_flip=False
                                 )
validation_data_gen=ImageDataGenerator(rescale=1./255)

train_batchsize=10
val_batchsize=15

train_generator=train_data_gen.flow_from_directory(train_dir,
                                                   target_size=(img_row,img_col),
                                                   batch_size=train_batchsize,
                                                   class_mode='categorical')
valid_generator=validation_data_gen.flow_from_directory(validation_dir,
                                                   target_size=(img_row,img_col),
                                                   batch_size=val_batchsize,
                                                   class_mode='categorical',
                                                  shuffle=False
                                                  )


                            
                                 

Found 300 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


### Training top layers

In [25]:
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping,ModelCheckpoint

checkpoint=ModelCheckpoint('200_model.h5',
                            monitor='val_loss',
                            mode='min',
                            save_best_only=True,
                            verbose=1
                          )
                           
early_stopping=EarlyStopping(monitor='val_loss',
                            min_delta=0,
                            patience=3,verbose=1,restore_best_weights=True)

callbacks=[early_stopping,checkpoint]

model_new.compile(loss='categorical_crossentropy',
                 optimizer=RMSprop(0.1),
                 metrics=['accuracy'] 
                 )

In [26]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
train_samples=300
validation_samples=100
epochs=10
batch_size=15

classifier=model_new.fit_generator(train_generator,
                                   steps_per_epoch=train_samples//batch_size,
                                  epochs=epochs,
                                  callbacks=callbacks,
                                   validation_data=valid_generator,
                                   validation_steps=validation_samples//batch_size
                                  )
                                   
                            
                                

Epoch 1/10
20/20 [==============================] - 78s 4s/step - loss: 4.2236 - accuracy: 0.1200 - val_loss: 2.2521 - val_accuracy: 0.1778

Epoch 00001: val_loss improved from inf to 2.25210, saving model to 200_model.h5
Epoch 2/10
20/20 [==============================] - 83s 4s/step - loss: 2.5363 - accuracy: 0.1550 - val_loss: 2.1836 - val_accuracy: 0.1529

Epoch 00002: val_loss improved from 2.25210 to 2.18359, saving model to 200_model.h5
Epoch 3/10
20/20 [==============================] - 75s 4s/step - loss: 2.4144 - accuracy: 0.1550 - val_loss: 2.6341 - val_accuracy: 0.2588

Epoch 00003: val_loss did not improve from 2.18359
Epoch 4/10
20/20 [==============================] - 76s 4s/step - loss: 2.3905 - accuracy: 0.1950 - val_loss: 4.1072 - val_accuracy: 0.1176

Epoch 00004: val_loss did not improve from 2.18359
Epoch 5/10
20/20 [==============================] - 70s 4s/step - loss: 2.4502 - accuracy: 0.2050 - val_loss: 1.8866 - val_accuracy: 0.0941

Epoch 00005: val_loss impro

In [ ]:
model_new.save("200_model.h5")
print("Saved model to disk")
